In [ ]:
from aisurveywriter.tasks import ReferenceExtractor
from aisurveywriter.core.llm_handler import LLMHandler
import aisurveywriter.core.file_handler as fh
from aisurveywriter.utils import get_all_files_from_paths
from aisurveywriter.core.pipeline import PaperPipeline
from aisurveywriter.core.paper import PaperData
import aisurveywriter.tasks as tks

import os
os.environ["GOOGLE_API_KEY"]=fh.read_credentials("../credentials.yaml")["google_key"]

llm = LLMHandler(model="gemini-2.0-pro-exp", model_type="google", temperature=0.5)
prompts = fh.read_yaml("../templates/prompt_config.yaml")
review = fh.read_yaml("../templates/review_config.yaml")

paper = PaperData.from_tex("../out/generated-revref.tex", subject="Langmuir and Langmuir-Blodgett Films")
# refs = tks.PaperReferencer(llm, bibdb_path="../out/generated-bibdb.bib", prompt="nada")
# with open(refs.bibdb_path, "r", encoding="utf-8") as f:
#     bibdb = f.read()
# refs._dump_used_bib(paper, bibdb_content=bibdb, save_path="../out/generated-used.bib")

pipe = PaperPipeline([
    ("Read Paper", tks.DeliverTask(paper)),
    ("Abstract and Title", tks.PaperRefiner(llm, prompt=prompts["refine_prompt"], cooldown_sec=70)),
    ("ReviewTex", tks.TexReviewer(llm, review["tex_review_prompt"])),
    ("SaveFinal", tks.PaperSaver("../out/generated.tex", "../templates/paper_template.tex", find_bib_pattern=None)),
])
pipe.run()